# Testing the likelihood external file

To start playing with this notebook, install the ```Likelihood Package``` and its modules according to README.
Alternatively you may be ready to run if ```Cobaya``` and ```CAMB/CLASS``` are installed.

Cobaya documentation: https://cobaya.readthedocs.io/en/latest/

In [1]:
#General imports
import numpy as np
import matplotlib.pyplot as plt

### Specific imports

In [2]:
#Import cobaya -need to be installed
import cobaya
#Import external loglike from the Likelihood Package within cobaya interface module
from likelihood.cobaya_interface import loglike

### Step 1: create your dictionary with general specifications

In ```COBAYA```, 'ini files' are 'yaml files', which are in reality understood by Cobaya as python dictionaries. I have added comments explaining each of the lines of the dictionary.


In [5]:
#ATTENTION: CHANGE THIS TO YOUR LOCAL PATH where your external codes are installed: CAMB, polychord, likelihoods...
modules_path = "/data2/cobaya_modules/"

# Info is the 'ini file' dictionary. 
# You can translate this info-dictionary into a yaml file and run: cobaya-run info.yaml such as CosmoMC like

info = {
    #Which parameters would you like to sample?
    'params': {
        # Fixed
        'pivot_scalar': 0.002,
        'ombh2': 0.022, 'omch2': 0.12, 'H0': 68, 'tau': 0.07,
        'mnu': 0.06, 'nnu': 3.046, 'num_massive_neutrinos': 1,
        'ns': 0.9674,
        #To be passed to euclid which likelihood to use (preliminary)
        # 1: shear
        # 2: spec
        # 12: both
        'like_selection': 12,
        # Sampled - just as an example we assume we will be sampling over A_s
        'As': 2.1e-9},
    #Which theory code you want to use? HERE CAMB
    'theory': {'camb': {'stop_at_error': True}},
    #Which sample do you want to use? HERE I use MCMC for testing
    'sampler': {'mcmc': None},  
    #Where have you installed your modules (i.e: CAMB, polychord, likelihoods...)
    'modules': modules_path,
    #Where are the results going to be stored?
    'output': 'chains/my_euclid_experiment',
    #Likelihood: we load the likelihood as an external function
    'likelihood': {'euclid': "import_module('likelihood.cobaya_interface').loglike"},
    }

### Step 2: Call the 'model' function of ```COBAYA```

In principle, we could run directly cobaya with the above dictionary and start producing chains. However, for testing purposes, the 'model' wrapper is created, which allows to enter directly to theory code's quantities. It works similarly as an 'evaluator'.

I use it here to give a feeling of what ```COBAYA``` is doing internally. Advice: good for debbuging too.

In [6]:
# Model wrapper of cobaya
from cobaya.model import get_model
model = get_model(info)

[model] *WARNING* Ignored blocks/options: ['sampler', 'output']
[prior] *WARNING* No sampled parameters requested! This will fail for non-mock samplers.
[euclid] Initialised external likelihood.
[camb] Importing *local* CAMB from /data2/cobaya_modules/code/CAMB
[likelihood] The theory code will compute the following products, requested by the likelihoods: ['Pk_interpolator', 'comoving_radial_distance', 'H', 'fsigma8']


In [12]:
# Which theory requirements is the likelihood asking to the Boltzman Solver?
print(model.likelihood.theory.requested())

{'Pk_interpolator': {'z': array([0.        , 0.27777778, 0.55555556, 0.83333333, 1.11111111,
       1.38888889, 1.66666667, 1.94444444, 2.22222222, 2.5       ]), 'k_max': 0.2, 'vars_pairs': {('delta_tot', 'delta_tot')}, 'extrap_kmax': 2, 'nonlinear': False}, 'comoving_radial_distance': {'z': array([0.        , 0.27777778, 0.55555556, 0.83333333, 1.11111111,
       1.38888889, 1.66666667, 1.94444444, 2.22222222, 2.5       ])}, 'H': {'z': array([0.        , 0.27777778, 0.55555556, 0.83333333, 1.11111111,
       1.38888889, 1.66666667, 1.94444444, 2.22222222, 2.5       ])}, 'fsigma8': {'z': array([0.        , 0.27777778, 0.55555556, 0.83333333, 1.11111111,
       1.38888889, 1.66666667, 1.94444444, 2.22222222, 2.5       ])}}


In [13]:
# To execute the likelihood, you need to make an evaluation of the posterior
# Attention: this chi2 for the moment, does not make sense yet
model.logposterior({})

logposterior(logpost=34206.19192968892, logpriors=[0], loglikes=array([34206.19192969]), derived=[])

### Step 3: Cosmology module and ```COBAYA```

In principle, the cobaya interface only pass cosmology parameters (for instance $\Omega_m$, $\Omega_b$, $H$) and ask for theory requirements from the boltzmann solver. Internally, the likelihood module uses the cosmology class to deal with the different GCspec, GCphot, WL recipes.

In [14]:
#Import Cosmology module from the Likelihood Package to play with cosmological quantities
from likelihood.cosmo.cosmology import Cosmology

In [17]:
# Some of the theory needs require extra info (redshift, ks)...
z_min = 0.0
z_max = 2.5
z_samp = 10
z_win = np.linspace(z_min, z_max, z_samp)

In [18]:
# Cobaya_interface save the cosmology parameters and the cosmology requirements 
# from CAMB/CLASS via COBAYA to the cosmology class

# This dictionary collects info from cobaya
theory_dic = {'H0': model.parameterization.constant_params()['H0'],
                  'omch2': model.parameterization.constant_params()['omch2'],
                  'ombh2': model.parameterization.constant_params()['ombh2'],
                  'mnu': model.parameterization.constant_params()['mnu'],
                  'comov_dist': model.likelihood.theory.get_comoving_radial_distance(z_win),
                  'H': model.likelihood.theory.get_H(z_win),
                  'Pk_interpolator': model.likelihood.theory.get_Pk_interpolator(),
                  'Pk_delta': None,
                  'fsigma8': None,
                  }
theory_dic['Pk_delta'] = (theory_dic['Pk_interpolator']
                              ['delta_tot_delta_tot'])
theory_dic['fsigma8'] = model.likelihood.theory.get_fsigma8(z_win)

In [22]:
# Initialize cosmology class from likelihood.cosmo.cosmology
# By default: LCDM
cosmology = Cosmology()
print(cosmology.cosmo_dic)

{'H0': 67.5, 'omch2': 0.122, 'ombh2': 0.022, 'mnu': 0.06, 'comov_dist': None, 'H': None, 'Pk_interpolator': None, 'Pk_delta': None, 'fsigma8': None, 'zk': None, 'b_gal': None, 'sigma_8': None}


In [24]:
# the cobaya_interface.py save cobaya theory dictionary as above into cosmology.cosmo_dic
cosmology.cosmo_dic = theory_dic
print(cosmology.cosmo_dic)

{'H0': 68, 'omch2': 0.12, 'ombh2': 0.022, 'mnu': 0.06, 'comov_dist': array([   0.        , 1142.9773538 , 2121.38122264, 2949.39932325,
       3650.57669895, 4248.45789676, 4763.09734232, 5210.53588629,
       5603.31481708, 5951.19577302]), 'H': array([ 68.        ,  78.57533123,  92.56648293, 109.50397342,
       128.94006714, 150.51786245, 173.9690815 , 199.09369509,
       225.74048994, 253.79257143]), 'Pk_interpolator': {'delta_tot_delta_tot': <camb.results.CAMBdata.get_matter_power_interpolator.<locals>.PKInterpolator object at 0x7fc11c7ce710>}, 'Pk_delta': <camb.results.CAMBdata.get_matter_power_interpolator.<locals>.PKInterpolator object at 0x7fc11c7ce710>, 'fsigma8': array([0.40225215, 0.44653764, 0.44818458, 0.42791323, 0.39935048,
       0.36939237, 0.34099903, 0.31520664, 0.29219735, 0.27180789])}


In [32]:
# Cosmology class calculates other cosmology quantities needed by GC, WL...
# Growth factor
D=cosmology.growth_factor(2, 0.002)
print('D(2, 0.002) =', D)
# Growth rate
f=cosmology.growth_rate(z_win, 0.002)
print('f(z, 0.002) =', f)

D(2, 0.002) = 0.4190851648999532
f(z, 0.002) = [0.13581661 0.18509345 0.21576715 0.23607684 0.24916947 0.25764292
 0.26322966 0.26700238 0.26961407 0.29182622]
